# Loading Images and Masks from DICOMs

In [1]:
from pathlib import Path

from imgtools.logging import logger
from imgtools.ops.input_classes import ImageMaskInput, ImageMaskModalities

logger.setLevel("ERROR")

/home/gpudual/bhklab/radiomics/Projects/med-imagetools/.pixi/envs/dev/lib/python3.12/site-packages/SimpleITK/extra.py:466: SyntaxWarning: invalid escape sequence '\s'
  """Blurs an image by separable convolution with discrete


## Getting Started

You need the following at minimum:
- Path to the directory containing the DICOM files
- Establish which Image and Mask modalities you want to use:
Combinations are:  

| Image Modality | Mask Modality |
|----------------|---------------|
| CT             | RTSTRUCT      |
| MR             | RTSTRUCT      |
| CT             | SEG      |
| MR             | SEG      |

For this tutorial we will download two datasets:


In [2]:
# Define the path to the data
testdata = Path("testdata")
# for this tutorial we will use some test image data
datasets_name = ["NSCLC-Radiomics", "Vestibular-Schwannoma-SEG"]

In [3]:
%%capture 
# download data using the imgtools cli
for ds in datasets_name:
    if not (testdata / f"{ds}.tar.gz").exists():
        print(f"Downloading {ds} dataset")
        !imgtools testdata -a {ds}.tar.gz {testdata.absolute()}


# Setting up the loaders with `ImageMaskInput`

## Vestibular-Schwannoma-SEG

has MR as scan and RTSTRUCT as mask:


| Patient ID           | Modality  | Number of Series    |
|----------------------|-----------|---------------------|
| VS-SEG-001           | MR        | 2                   |
| VS-SEG-001           | RTSTRUCT  | 2                   |
| VS-SEG-002           | MR        | 2                   |
| VS-SEG-002           | RTSTRUCT  | 2                   |

In [4]:
vs_seg = ImageMaskInput(
  dir_path=testdata / datasets_name[1], # "Vestibular-Schwannoma-SEG"
  modalities=ImageMaskModalities.MR_RTSTRUCT
)

print(vs_seg)

ImageMaskInput<
	num_cases=4,
	dataset_name='Vestibular-Schwannoma-SEG',
	modalities=<MR,RTSTRUCT>,
	output_streams=['MR', 'RTSTRUCT_MR'],
	series_col_names=['series_MR', 'series_RTSTRUCT_MR'],
>


## NSCLC-Radiomics

has CT as scan and BOTH RTSTRUCT and SEG as masks.

| Patient ID           | Modality  | Number of Series    |
|----------------------|-----------|---------------------|
| LUNG1-001            | CT        | 1                   |
| LUNG1-001            | RTSTRUCT  | 1                   |
| LUNG1-001            | SEG       | 1                   |
| LUNG1-002            | CT        | 1                   |
| LUNG1-002            | RTSTRUCT  | 1                   |
| LUNG1-002            | SEG       | 1                   |

#### Use the `ImageMaskModalities` enum to specify the modalities you want to use

***NSCLC-Radiomics CT & RTSTRUCT***

In [5]:
nsclsc_rtstruct = ImageMaskInput(
    dir_path=testdata / datasets_name[0],
    modalities=ImageMaskModalities.CT_RTSTRUCT
)
nsclsc_rtstruct

ImageMaskInput<
	num_cases=2,
	dataset_name='NSCLC-Radiomics',
	modalities=<CT,RTSTRUCT>,
	output_streams=['CT', 'RTSTRUCT_CT'],
	series_col_names=['series_CT', 'series_RTSTRUCT_CT'],
>

***NSCLC-Radiomics CT & SEG***

In [6]:
nsclsc_seg = ImageMaskInput(
    dir_path=testdata / datasets_name[0],
    modalities=ImageMaskModalities.CT_SEG
)
nsclsc_seg

ImageMaskInput<
	num_cases=2,
	dataset_name='NSCLC-Radiomics',
	modalities=<CT,SEG>,
	output_streams=['CT', 'SEG'],
	series_col_names=['series_CT', 'series_SEG'],
>

# Using the Input Datasets



In [7]:
# List the case IDs (subject IDs)
nsclsc_rtstruct.keys()

['0_LUNG1-002', '1_LUNG1-001']

## Load a case


`Image-Mask` pairs are loaded as as `imgtools.ImageMask` object which is a `NamedTuple` with the following fields:

- `scan` - The image data (`imgtools.Scan` object)
- `mask` - The mask data (`imgtools.Segmentation` object)


> !!NOTE: By default, data is `lazy-loaded` and only the metadata is loaded. The actual data is loaded when requested.

We provide 4 different ways to access the data:


### Access by `Case ID`

both the `__getitem__` and `__call__` methods can be used to access the data by `Case ID`
and are equivalent.

In [ ]:
# > ‼️NOTE `StructureSets` are automatically converted to `imgtools.Segmentation` objects
# instance `__call__` method to access the data
nsclsc_rtstruct('0_LUNG1-002')

ImageMask(scan=Scan<
	BodyPartExamined=LUNG
	SliceThickness=3.00000
	PatientPosition=HFS
	Manufacturer=CMS, Inc.
	RescaleSlope=1
	ManufacturerModelName=XiO
	PixelSize=('0.9770', '0.9770', '3.00000')
	KVP=None, size=(512, 512, 111)
	spacing=(0.977, 0.977, 3.0)
	origin=(-250.112, -250.112, -133.4)
	direction=(1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0)
>, mask=<Segmentation with ROIs: {'Esophagus': 1, 'GTV-1': 2, 'Heart': 3, 'Lung-Left': 4, 'Lung-Right': 5, 'Spinal-Cord': 6}>)

In [ ]:
# unpack the tuple to get the image and mask
scan, mask = nsclsc_rtstruct['0_LUNG1-002']
mask

<Segmentation with ROIs: {'Esophagus': 1, 'GTV-1': 2, 'Heart': 3, 'Lung-Left': 4, 'Lung-Right': 5, 'Spinal-Cord': 6}>

In [12]:
# alternatively, can assign to one object and access the scan and mask
scan_mask = nsclsc_rtstruct['0_LUNG1-002']

print(repr(scan_mask.scan))
print(repr(scan_mask.mask))

Scan<
	BodyPartExamined=LUNG
	SliceThickness=3.00000
	PatientPosition=HFS
	Manufacturer=CMS, Inc.
	RescaleSlope=1
	ManufacturerModelName=XiO
	PixelSize=('0.9770', '0.9770', '3.00000')
	KVP=None, size=(512, 512, 111)
	spacing=(0.977, 0.977, 3.0)
	origin=(-250.112, -250.112, -133.4)
	direction=(1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0)
>
<Segmentation with ROIs: {'Esophagus': 1, 'GTV-1': 2, 'Heart': 3, 'Lung-Left': 4, 'Lung-Right': 5, 'Spinal-Cord': 6}>


### Access by integer index

To easily access the data, without knowing the `Case ID`, you can use the integer index. 

i.e `loader[0]` will return the first case in the dataset.

In [10]:
# by case ID or index
scan, mask  = nsclsc_seg[0]
mask

<Segmentation with ROIs: {'label_1': 1}>

### Iterating over the loader

Example for MR & RTSTRUCT example

In [8]:
for i, (scan, mask) in enumerate(vs_seg):
    print(f"CASE-{i:<2}: {type(scan)=}, {type(mask)=}")

CASE-0 : type(scan)=<class 'imgtools.modules.scan.Scan'>, type(mask)=<class 'imgtools.modules.segmentation.Segmentation'>
CASE-1 : type(scan)=<class 'imgtools.modules.scan.Scan'>, type(mask)=<class 'imgtools.modules.segmentation.Segmentation'>
CASE-2 : type(scan)=<class 'imgtools.modules.scan.Scan'>, type(mask)=<class 'imgtools.modules.segmentation.Segmentation'>
CASE-3 : type(scan)=<class 'imgtools.modules.scan.Scan'>, type(mask)=<class 'imgtools.modules.segmentation.Segmentation'>
